In [1]:
import os
import numpy as np
from utils.visualize import *

import json
from tqdm import tqdm
import open3d as o3d
import transforms3d
from utils.preprocess import path_dict
from utils.read_json import *


In [2]:
scans = os.listdir("/scannet/crop_scan2cad_filter/data")

chairs = []
tables = []
for scan in scans:
    if scan[:12] == "scene0015_00" and scan.split('.')[2] == "03001627":
        chairs.append(scan)
    if scan[:12] == "scene0015_00" and scan.split('.')[2] == "04379243":
        tables.append(scan)
        
print(chairs)
print(tables)

['scene0015_00.9.03001627.e6328c1bb6b194f262e682c9809bff14.npy', 'scene0015_00.8.03001627.e6328c1bb6b194f262e682c9809bff14.npy', 'scene0015_00.7.03001627.e6328c1bb6b194f262e682c9809bff14.npy', 'scene0015_00.15.03001627.1be38f2624022098f71e06115e9c3b3e.npy', 'scene0015_00.25.03001627.1be38f2624022098f71e06115e9c3b3e.npy', 'scene0015_00.19.03001627.1be38f2624022098f71e06115e9c3b3e.npy', 'scene0015_00.18.03001627.1be38f2624022098f71e06115e9c3b3e.npy', 'scene0015_00.17.03001627.1be38f2624022098f71e06115e9c3b3e.npy', 'scene0015_00.1.03001627.e6328c1bb6b194f262e682c9809bff14.npy', 'scene0015_00.14.03001627.1be38f2624022098f71e06115e9c3b3e.npy', 'scene0015_00.0.03001627.73f9aa75944ecf0b9debdd405104de8c.npy', 'scene0015_00.11.03001627.e6328c1bb6b194f262e682c9809bff14.npy', 'scene0015_00.10.03001627.e6328c1bb6b194f262e682c9809bff14.npy', 'scene0015_00.24.03001627.1be38f2624022098f71e06115e9c3b3e.npy', 'scene0015_00.20.03001627.1be38f2624022098f71e06115e9c3b3e.npy', 'scene0015_00.16.03001627.1be

In [3]:
scans = os.listdir("/scannet/crop_scan2cad")

chairs = []
tables = []
for scan in scans:
    if scan[:12] == "scene0015_00" and scan.split('.')[2] == "03001627":
        chairs.append(scan)
    if scan[:12] == "scene0015_00" and scan.split('.')[2] == "04379243":
        tables.append(scan)
        
print(chairs)
print(tables)

['scene0015_00.9.03001627.e6328c1bb6b194f262e682c9809bff14.npy', 'scene0015_00.8.03001627.e6328c1bb6b194f262e682c9809bff14.npy', 'scene0015_00.7.03001627.e6328c1bb6b194f262e682c9809bff14.npy', 'scene0015_00.15.03001627.1be38f2624022098f71e06115e9c3b3e.npy', 'scene0015_00.25.03001627.1be38f2624022098f71e06115e9c3b3e.npy', 'scene0015_00.19.03001627.1be38f2624022098f71e06115e9c3b3e.npy', 'scene0015_00.21.03001627.1be38f2624022098f71e06115e9c3b3e.npy', 'scene0015_00.22.03001627.1be38f2624022098f71e06115e9c3b3e.npy', 'scene0015_00.18.03001627.1be38f2624022098f71e06115e9c3b3e.npy', 'scene0015_00.17.03001627.1be38f2624022098f71e06115e9c3b3e.npy', 'scene0015_00.1.03001627.e6328c1bb6b194f262e682c9809bff14.npy', 'scene0015_00.14.03001627.1be38f2624022098f71e06115e9c3b3e.npy', 'scene0015_00.0.03001627.73f9aa75944ecf0b9debdd405104de8c.npy', 'scene0015_00.11.03001627.e6328c1bb6b194f262e682c9809bff14.npy', 'scene0015_00.10.03001627.e6328c1bb6b194f262e682c9809bff14.npy', 'scene0015_00.24.03001627.1be

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import argparse
from trainer.RetrievalTrainer import trainer
from datasets.ScannetDataset import ScannetDataset
from datasets.CategoryTestTimeDataset import *
from datasets.Reader import *

from test_time.FeatureExtractor import FeatureExtractor
from test_time.RetrievalModule import RetrievalModule

from utils.logger import logger
from utils.ckpts import load_checkpoint, save_checkpoint
from utils.retrieval import *
from utils.visualize import *
from utils.preprocess import random_rotation, load_norm_pc, apply_transform
from utils.symmetry import*
from utils.eval_pose import *
from utils.Info.Scan2cadInfo import Scan2cadInfo
from utils.Info.CADLib import *

from model import load_model, fc

from scipy.spatial.distance import pdist, squareform
from sklearn.cluster import KMeans

import MinkowskiEngine as ME

torch.manual_seed(0)
torch.cuda.manual_seed(0)

class Config():
    def __init__(self):

        self.root = "/scannet/ShapeNetCore.v2.PC15k"
        self.scan2cad_root = "/scannet/crop_scan2cad_filter/data"
        self.cad_root = "/scannet/ShapeNetCore.v2.PC15k"
        #self.catid = "04379243"
        self.catid = "03001627"
        self.voxel_size = 0.03
        self.dim = [1024, 512,  256]
        self.embedding = "conv1_max_embedding"
        #self.resume = "./ckpts/scannet_table_ret_max_sim0"
        self.resume = "./ckpts/scannet_ret_max_sim0"
        self.model = "ResUNetBN2C"
        self.model_n_out = 16
        self.normalize_feature = True
        self.conv1_kernel_size = 3
        self.bn_momentum = 0.05
        self.nn_max_n = 500
        self.lib_path = "./CadLib"
        self.scan2cad_dict = "/scannet/scan2cad_download_link/unique_cads.csv"
        self.annotation_dir = "/scannet/scan2cad_download_link"


config_chair = Config()
config_chair.catid = "03001627"
config_chair.resume = "./ckpts/scannet_ret_max_sim0"
config_table = Config()
config_table.catid = "04379243"
config_table.resume = "./ckpts/scannet_table_pose_001_usedLib_bestMatch_Best"#"./ckpts/scannet_table_ret_max_sim0"


In [1]:
def chamfer_gpu_1direction(pc0, pc1):

    pc0 = pc0[None,:,:]
    pc1 = pc1[:,None,:]
    delta = pc0 - pc1
    return delta.norm(dim=2).min(0)[0].mean()

In [39]:
feature_extractor_chair = FeatureExtractor(config_chair)
feature_extractor_table = FeatureExtractor(config_table)


Feature Extraction
Embedding: conv1_max_embedding
FC dim: [1024, 512, 256]
voxel size: 0.03
loading checkpoint from ./ckpts/scannet_ret_max_sim0
Model at epoch: 99
Feature Extraction
Embedding: conv1_max_embedding
FC dim: [1024, 512, 256]
voxel size: 0.03
loading checkpoint from ./ckpts/scannet_table_pose_001_usedLib_bestMatch_Best
Model at epoch: 94


In [40]:
retrieval_module_chair = RetrievalModule(config_chair, feature_extractor_chair, "scan2cad", update=True)
retrieval_module_table = RetrievalModule(config_table, feature_extractor_table, "scan2cad", update=True)

100%|██████████| 652/652 [00:01<00:00, 633.04it/s]

Finish Loading


Eval feature Index: 1/21
Eval feature Index: 11/21
Eval feature Index: 21/21


100%|██████████| 830/830 [00:01<00:00, 685.12it/s]

Finish Loading


Eval feature Index: 1/26
Eval feature Index: 11/26
Eval feature Index: 21/26


In [73]:
root = "/scannet/crop_scan2cad"
scene_id = "scene0653_00"
scans = os.listdir(root)

chairs = []
tables = []
for scan in scans:
    if scan[:12] == scene_id and scan.split('.')[2] == "03001627":
        chairs.append(scan)
    if scan[:12] == scene_id and scan.split('.')[2] == "04379243":
        tables.append(scan)
        
print(chairs)
print(tables)

save_dir = os.path.join("/zty-vol/results/scene_reconstruct", scene_id)
if not os.path.exists(save_dir):
    os.mkdir(save_dir)
    


['scene0653_00.9.03001627.60647c98b5fc96b13002761e7a3ba3bd.npy', 'scene0653_00.10.03001627.60647c98b5fc96b13002761e7a3ba3bd.npy', 'scene0653_00.11.03001627.60647c98b5fc96b13002761e7a3ba3bd.npy', 'scene0653_00.7.03001627.60647c98b5fc96b13002761e7a3ba3bd.npy', 'scene0653_00.5.03001627.60647c98b5fc96b13002761e7a3ba3bd.npy', 'scene0653_00.8.03001627.60647c98b5fc96b13002761e7a3ba3bd.npy', 'scene0653_00.21.03001627.7212bdfab35f72b23002761e7a3ba3bd.npy', 'scene0653_00.12.03001627.60647c98b5fc96b13002761e7a3ba3bd.npy']
['scene0653_00.15.04379243.24942a3b98d1bcb6a570c6c691c987a8.npy', 'scene0653_00.14.04379243.24942a3b98d1bcb6a570c6c691c987a8.npy', 'scene0653_00.13.04379243.24942a3b98d1bcb6a570c6c691c987a8.npy', 'scene0653_00.16.04379243.24942a3b98d1bcb6a570c6c691c987a8.npy', 'scene0653_00.25.04379243.24942a3b98d1bcb6a570c6c691c987a8.npy', 'scene0653_00.17.04379243.24942a3b98d1bcb6a570c6c691c987a8.npy']


In [74]:
visual_pcs = []
colors = []

for idx, name in enumerate(chairs):
    pc = np.load(os.path.join(root, name))
    offset = pc.mean(0)
    pc_center = pc - offset
    r = np.linalg.norm(pc_center, 2, 1).max()
    pc_center /= r
    #Jvisualize([pc_center], ["RED"])
    
    #feature extraction
    base_local_feat, base_global_feat, base_coords = feature_extractor_chair.process(pc_center)

    # retrieval
    ret_path = retrieval_module_chair.TopN(base_global_feat.detach().cpu().numpy())
    print(ret_path)
    ret_raw_pc = load_raw_pc(ret_path[0], 10000)
    
    norm_pc_offset = ret_raw_pc.mean(0)
    ret_pc = ret_raw_pc - norm_pc_offset
    norm_pc_r = np.max(np.linalg.norm(ret_pc,2,1))
    ret_pc = ret_pc/norm_pc_r
    
    ret_local_feat, ret_global_feat, ret_coords = feature_extractor_chair.process(ret_pc)
    
    #pose estimation
    k_nn=5
    max_corr = 0.20

    baseF, xyz0, posF, xyz1 = ret_local_feat, ret_coords, base_local_feat, base_coords

    """
    idx_0, idx_1 = find_kcorr(baseF, posF, k=k_nn, subsample_size=-1)

    source_pcd = xyz0[idx_0]
    target_pcd = xyz1[idx_1]

    T_est = registration_based_on_corr(source_pcd, target_pcd, max_corr)

    T_est = torch.from_numpy(T_est.astype(np.float32))
    """
    
    
    T_est = sym_pose(posF, xyz1, baseF, xyz0, 1)
    
    # alignment
    ali_ret_pc = apply_transform(ret_pc, np.linalg.inv(T_est))
    ali_cad = ali_ret_pc*r + offset
    
    visual_pcs += [pc, ali_cad]
    colors += ["RED", "GREEN"]
    
    objid = ret_path[0].split('/')[-1].split('.')[0]
    #print(objid)
    
    obj_dir = os.path.join(save_dir, objid+"_"+str(idx))
    if not os.path.exists(obj_dir):
        os.mkdir(obj_dir)
        
    np.save(os.path.join(obj_dir, "T_est.npy"), T_est)
    
    with open(os.path.join(obj_dir, "scale_trans.txt"), "w") as f:
        f.writelines(str(norm_pc_r)+'\n')
        for val in norm_pc_offset:
            f.writelines(str(val)+' ')
        f.writelines('\n')
        f.writelines(str(r)+'\n')
        for val in offset:
            f.writelines(str(val)+' ')
        
    
    
for idx, name in enumerate(tables):
    pc = np.load(os.path.join(root, name))
    offset = pc.mean(0)
    pc_center = pc - offset
    r = np.linalg.norm(pc_center, 2, 1).max()
    pc_center /= r
    #Jvisualize([pc_center], ["RED"])
    
    #feature extraction
    base_local_feat, base_global_feat, base_coords = feature_extractor_table.process(pc_center)

    # retrieval
    ret_path = retrieval_module_table.TopN(base_global_feat.detach().cpu().numpy(), 1)
    print(ret_path)

    ret_raw_pc = load_raw_pc(ret_path[0], 10000)
    
    norm_pc_offset = ret_raw_pc.mean(0)
    ret_pc = ret_raw_pc - norm_pc_offset
    norm_pc_r = np.max(np.linalg.norm(ret_pc,2,1))
    ret_pc = ret_pc/norm_pc_r
    ret_local_feat, ret_global_feat, ret_coords = feature_extractor_table.process(ret_pc)
    
    #pose estimation
    k_nn=5
    max_corr = 0.20

    baseF, xyz0, posF, xyz1 = ret_local_feat, ret_coords, base_local_feat, base_coords

    """
    idx_0, idx_1 = find_kcorr(baseF, posF, k=k_nn, subsample_size=-1)

    source_pcd = xyz0[idx_0]
    target_pcd = xyz1[idx_1]

    T_est = registration_based_on_corr(source_pcd, target_pcd, max_corr)

    T_est = torch.from_numpy(T_est.astype(np.float32))
    """
    
    T_est = sym_pose(posF, xyz1, baseF, xyz0, 2)
    
    # alignment
    ali_ret_pc = apply_transform(ret_pc, np.linalg.inv(T_est))
    ali_cad = ali_ret_pc*r + offset
    
    visual_pcs += [pc, ali_cad]
    colors += ["RED", "GREEN"]

    objid = ret_path[0].split('/')[-1].split('.')[0]
    #print(objid)
    
    obj_dir = os.path.join(save_dir, objid+"_"+str(idx))
    if not os.path.exists(obj_dir):
        os.mkdir(obj_dir)
        
    np.save(os.path.join(obj_dir, "T_est.npy"), T_est)
    
    with open(os.path.join(obj_dir, "scale_trans.txt"), "w") as f:
        f.writelines(str(norm_pc_r)+'\n')
        for val in norm_pc_offset:
            f.writelines(str(val)+' ')
        f.writelines('\n')
        f.writelines(str(r)+'\n')
        for val in offset:
            f.writelines(str(val)+' ')
    
Jvisualize(visual_pcs, colors)

['/scannet/ShapeNetCore.v2.PC15k/03001627/train/6b64af50e21c0006f91663a74ccd2338.npy']
['/scannet/ShapeNetCore.v2.PC15k/03001627/train/6f36520144753550f91663a74ccd2338.npy']
['/scannet/ShapeNetCore.v2.PC15k/03001627/train/7212bdfab35f72b23002761e7a3ba3bd.npy']
['/scannet/ShapeNetCore.v2.PC15k/03001627/train/6fde09bdd613f6e6492d9da2668ec34c.npy']
use ransac
['/scannet/ShapeNetCore.v2.PC15k/03001627/train/f4cec47ced59d95a3002761e7a3ba3bd.npy']
use ransac
['/scannet/ShapeNetCore.v2.PC15k/03001627/train/6b64af50e21c0006f91663a74ccd2338.npy']
use ransac
['/scannet/ShapeNetCore.v2.PC15k/03001627/train/8a948db5f12d02af492d9da2668ec34c.npy']
['/scannet/ShapeNetCore.v2.PC15k/03001627/train/6b64af50e21c0006f91663a74ccd2338.npy']
use ransac
['/scannet/ShapeNetCore.v2.PC15k/04379243/test/4bbf789edb243cafc955e5ed03ef3a2f.npy']
['/scannet/ShapeNetCore.v2.PC15k/04379243/train/24942a3b98d1bcb6a570c6c691c987a8.npy']
['/scannet/ShapeNetCore.v2.PC15k/04379243/train/968714b674baa3152cf0938654a53e55.npy']


In [44]:
def sym_pose(baseF, xyz0, posF, xyz1, pos_sym):
    #print(pos_sym)

    try:
        if pos_sym >= 2:
            base_mask1, base_mask2, base_mask3, base_mask4 = symmetric_cut4(baseF, xyz0, 4, max_sample=100)

            pos_mask1, pos_mask2, pos_mask3, pos_mask4 = symmetric_cut4(posF, xyz1, 4, max_sample=100)

            base_masks = [base_mask1, base_mask2, base_mask3, base_mask4]
            pos_masks = [pos_mask1, pos_mask2, pos_mask3, pos_mask4]

        else:
            #print(pos_sym)
            base_mask1, base_mask2 = symmetric_cut4(baseF, xyz0, 2, max_sample=100)

            pos_mask1, pos_mask2 = symmetric_cut4(posF, xyz1, 2, max_sample=100)

            base_masks = [base_mask1, base_mask2]
            pos_masks = [pos_mask1, pos_mask2]

            #Jvisualize([xyz0[base_mask1], xyz0[base_mask2]], ["RED", "GREEN"])


        chamf_dist_best = 100

        for _ in range(len(base_masks)):

            pcsA = [xyz0[base_mask] for base_mask in base_masks]
            pcsB = [xyz1[pos_mask] for pos_mask in pos_masks]

            featsA = [baseF[base_mask] for base_mask in base_masks]
            featsB = [posF[pos_mask] for pos_mask in pos_masks]

            xyzA_corrs, xyzB_corrs = split_corr(pcsA, pcsB, featsA, featsB, k_nn, subsample_size=-1)

            T_est = registration_based_on_corr(xyzA_corrs, xyzB_corrs, max_corr)

            T_est = torch.from_numpy(T_est.astype(np.float32))

            #t_loss, r_loss = eval_pose(T_est, base_T, pos_T, axis_symmetry=max(base_sym, pos_sym))

            chamf_dist = chamfer_gpu_1direction(apply_transform(xyz0, T_est).cuda(), xyz1.cuda())
            #avg_dist = np.linalg.norm(apply_transform(xyzA_corrs, T_est) - xyzB_corrs, axis=1).mean()

            rot_item = pos_masks.pop(0)
            pos_masks.append(rot_item)

            #print(chamf_dist)

            if chamf_dist_best > chamf_dist:

                chamf_dist_best = chamf_dist
                T_est_best = T_est


        if pos_sym >= 2:
            base_masks = [base_mask1, base_mask2, base_mask3, base_mask4]
            pos_masks = [pos_mask1, pos_mask4, pos_mask3, pos_mask2]
        else:

            base_masks = []
            pos_masks = []


        for _ in range(len(base_masks)):

            pcsA = [xyz0[base_mask] for base_mask in base_masks]
            pcsB = [xyz1[pos_mask] for pos_mask in pos_masks]

            featsA = [baseF[base_mask] for base_mask in base_masks]
            featsB = [posF[pos_mask] for pos_mask in pos_masks]

            xyzA_corrs, xyzB_corrs = split_corr(pcsA, pcsB, featsA, featsB, k_nn, subsample_size=-1)

            T_est = registration_based_on_corr(xyzA_corrs, xyzB_corrs, max_corr)

            T_est = torch.from_numpy(T_est.astype(np.float32))

            #t_loss, r_loss = eval_pose(T_est, base_T, pos_T, axis_symmetry=max(base_sym, pos_sym))

            chamf_dist = chamfer_gpu_1direction(apply_transform(xyz0, T_est).cuda(), xyz1.cuda())
            #avg_dist = np.linalg.norm(apply_transform(xyzA_corrs, T_est) - xyzB_corrs, axis=1).mean()

            rot_item = pos_masks.pop(0)
            pos_masks.append(rot_item)

            #print(chamf_dist)

            if chamf_dist_best > chamf_dist:

                chamf_dist_best = chamf_dist
                T_est_best = T_est

        idx_0, idx_1 = find_kcorr(baseF, posF, k=k_nn, subsample_size=-1)

        source_pcd = xyz0[idx_0]
        target_pcd = xyz1[idx_1]

        T_est = registration_based_on_corr(source_pcd, target_pcd, max_corr)

        T_est = torch.from_numpy(T_est.astype(np.float32))
        chamf_dist = chamfer_gpu_1direction(apply_transform(xyz0, T_est).cuda(), xyz1.cuda())
        #avg_dist = np.linalg.norm(apply_transform(xyzA_corrs, T_est) - xyzB_corrs, axis=1).mean()  

        #print("ransac:")
        #print(chamf_dist)

        if chamf_dist_best > chamf_dist:

            chamf_dist_best = chamf_dist
            T_est_best = T_est
            print("use ransac")

    
    except:
    
        idx_0, idx_1 = find_kcorr(baseF, posF, k=k_nn, subsample_size=-1)

        source_pcd = xyz0[idx_0]
        target_pcd = xyz1[idx_1]

        T_est = registration_based_on_corr(source_pcd, target_pcd, max_corr)

        T_est = torch.from_numpy(T_est.astype(np.float32))

    return T_est
    

In [56]:
Jvisualize([pc],["RED"])

JVisualizer with 1 geometries

scene0314_00
scene0690_00
scene0474_00
scene0355_00
scene0653_00